In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install unstructured
!pip install faiss-cpu
!pip install sentence-transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Langchain Basic

In [1]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:480: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [5]:
# Use a pipeline for later
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing HuggingFacePipeline from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing LLMChain from langchain root module is no longer supported.
  warnings.warn(


In [6]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [7]:
name = llm("I want to open a resturant for Indian Food. Suggest a fancy name for that.")

print(name)


Thanks for sharing your thoughts on this topic. I am looking to open a restaurant that serves Indian food, and I think a fancy name could help to attract customers. Here are a few suggestions for a fancy name for an Indian restaurant:
1. Tandoori Nights: This name suggests a sophisticated and elegant atmosphere, perfect for a fine dining experience.
2. Spice Route: This name evokes the idea of a journey through the flavors of India, which could appeal to adventurous foodies.
3. Maharaja's Palace: This name conveys a sense of luxury and grandeur, which could help to create a regal atmosphere in the restaurant.
4. Curry Leaf: This name is simple and straightforward, yet it also suggests a focus on the rich flavors of Indian cuisine.
5. Mumbai Street Food Co.: This name suggests a fun and casual atmosphere, while also highlighting the authenticity of the food.
6. Tikka House: This name suggests a restaurant that specializes in tandoori dishes, which are a popular part of Indian cuisine.


In [8]:
menu = llm("Suggest some menu items for an indian resturant.")

print(menu)


Most popular indian restaurant dishes are:
1. Tandoori chicken: marinated in yogurt and spices and cooked in a tandoor clay oven.
2. Naan bread: a leavened, butter-topped flatbread popular in northern India.
3. Biryani: a rice dish made with aromatic spices, basmati rice, and meat or vegetables.
4. Palak paneer: a creamy spinach dish made with paneer (Indian cheese) and spices.
5. Tikka masala: a popular dish from the Punjab region, made with marinated chicken cooked in a creamy tomato sauce.
6. Samosas: crispy fried or baked pastries filled with spiced potatoes, peas, and onions.
7. Gulab jamun: deep-fried dumplings soaked in a sweet syrup made with rosewater and cardamom.
8. Korma: a mild and creamy dish made with yogurt, nuts, and spices, typically served with chicken or lamb.
9. Rogan josh: a lamb dish from the Kashmir region, made with yogurt, spices, and saffron.
10. Chole bhature: a spicy chickpea curry served with a deep-fried bread called bhature.
These are just a few example

## Making use of Langchain

In [2]:
import os
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import langchain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing HuggingFacePipeline from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing PromptTemplate from langchain root module is no longer supported.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing LLMChain from langchain root module is no longer supported.
  warnings.warn(


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                            #  torch_dtype=torch.float16,
                                             use_auth_token=True,
                                             )

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                # torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:480: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [12]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()
len(data)

2

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

chunks = text_splitter.split_documents(data)
len(chunks)

41

In [14]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'})

In [15]:
vectorindex_openai = FAISS.from_documents(chunks, embeddings)

In [16]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorindex_openai.as_retriever())

In [17]:
query = "what is the price of Tiago iCNG?"
# query = "what are the main features of punch iCNG?"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [1:chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?"
}
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nTags:\n\n#Business\n\n#Companies\n\nfirst published: Aug 4, 2023 02:17 pm\n\nbusiness news,\n\nSensex, and\n\nNif

Token indices sequence length is longer than the specified maximum sequence length for this model (1924 > 1024). Running this sequence through the model will result in indexing errors


[chain/start] [1:chain:RetrievalQAWithSourcesChain > 3:chain:MapReduceDocumentsChain > 9:chain:LLMChain] Entering Chain run with input:
{
  "question": "what is the price of Tiago iCNG?",
  "summaries": "Content: \nTiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh.\nSource: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n\nContent: \n\nTata Motors launches Punch iCNG, price starts at Rs 7.1 lakh.\n\nThe price of Tiago iCNG lacost?\nSource: https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html\n\nContent: \n* Punch iCNG\n* Rs 7.1 lakh\n* Rs 9.68 lakh\n* ex-showroom, Delhi\n* Tata Motors\n* CNG variant\n* micro SUV\n* twin-cylinder technology\n* safety features\n* micro-switch\n* thermal incident protection\n* automatic projector headlamps\n* LED DRLs\n* 16-inch diamond cut alloy wheels\n* 7-inch infotainment system\n* Android Auto\n* Apple Carplay 

{'answer': ' The price of Tiago iCNG is Rs 7.1 lakh.\n', 'sources': ''}